# Lyft: データの理解とEDA

### クレジット:

**https://www.kaggle.com/t3nyks/lyft-working-with-map-api**<br>
**https://www.kaggle.com/jpbremer/lyft-scene-visualisations**<br>
**https://www.kaggle.com/pestipeti/pytorch-baseline-train**

注釈：
本記事は https://www.kaggle.com/nxrprime/lyft-understanding-the-data-baseline-model を日本語に訳したものになります。
内容の誤りや誤訳については予めご容赦ください。

この新しいLyftのコンペは、参加者である私たちに、自動車や自転車、歩行者などの動きを予測して、自動運転車を支援することを課題としています。昨年のコンペでは、ストップサインなどの立体物を検知して自動運転車に認識方法を教えるという課題がありましたが、これは一歩前進しています。

これは明らかに**最大の交通エージェントのモーションデータのコレクションです。**ファイルはPythonで.zarrファイル形式で保存されています。トレーニングZARRの中には、エージェント、エージェント用マスク、フレームとシーン、信号機があります。

テスト用ZARRはほぼ同じフォーマットですが、データマスクが除外されているだけです。

## データの利用を開始する

In [ ]:
!pip install --upgrade pip
!pip install pymap3d==2.1.0
!pip install -U l5kit

待ってください！可視化とそれに伴うすべての処理に入る前に、短いYouTubeのビデオを見て、自律走行車の操作の主題について知りませんか？

In [ ]:
from IPython.display import HTML
HTML('<center><iframe width="700" height="400" src="https://www.youtube.com/embed/tlThdr3O5Qo?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe></center>')

この車は、ドライバーが直面する通常の課題をさりげなくこなしているように見えます。それも驚くほどの精度です。ここでは、このようなことを実現するために、外部要因の動きを予測し、それに基づいて自動運転車の動きの経路を予測することが課題となっています***このような外部要因の動きを予測するためには、後述するように様々なアプローチがありますが、とりあえず飛び込んでみましょう。

ここでは、データセットとその内容についての簡単なFAQを紹介します。

**データセットの構造は？**<br>
データセットは以下のように構成されています。
```
aerial_map
scenes
semantic_map
```

ここでは、各シーンには、複数の外部車両の動きとそれに対応する自動運転車の動きに関する情報が約1分程度含まれています。

データ一覧:
```
sample.zarr
test.zarr
train.zarr
validate.zarr
```

今、このZARRフォーマットは、私は参加者のほとんどがこれらを使用したことがないと思っているので、少し興味深いです。これらはNumPyと非常に多くの相互運用性があり、Lyftレベル5キットはまた、データの処理を処理するための簡単な方法を提供していますので、心配しないでください。もちろん、その過程でPandas DataFrameを使用する方法もいくつかあるかもしれませんが、ここではLightGBMを用います。

train.zarrにはエージェント、エージェント用のマスク、フレーム、シーン、信号機の顔が含まれていますが、これについては後ほど詳しく説明します。

リフトのレベル5キットとそれに付属するすべてのものをインポートすることができるようになりました。

In [ ]:
import l5kit, os
from l5kit.rasterization import build_rasterizer
from l5kit.configs import load_config_data
from l5kit.visualization import draw_trajectory, TARGET_POINTS_COLOR
from l5kit.geometry import transform_points
from tqdm import tqdm
from collections import Counter
from l5kit.data import PERCEPTION_LABELS
from prettytable import PrettyTable
# データ環境変数を設定する
os.environ["L5KIT_DATA_FOLDER"] = "../input/lyft-motion-prediction-autonomous-vehicles"
# 取得設定
cfg = load_config_data("../input/lyft-config-files/visualisation_config.yaml")
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

では、設定データを見てみましょう。これには、エージェントに関するメタデータ、合計時間、1シーンあたりのフレーム数、シーン時間、フレーム数が含まれます。

In [ ]:
from l5kit.data import ChunkedDataset, LocalDataManager
from l5kit.dataset import EgoDataset, AgentDataset
dm = LocalDataManager()
dataset_path = dm.require(cfg["val_data_loader"]["key"])
zarr_dataset = ChunkedDataset(dataset_path)
zarr_dataset.open()
print(zarr_dataset)

しかし現在は、シーンを見て深く分析する時です。理論的には私たちのために重労働をしてくれる気の利いた小さなデータローダを作ることができます。

In [ ]:
import numpy as np
from IPython.display import display, clear_output
import PIL
 
cfg["raster_params"]["map_type"] = "py_semantic"
rast = build_rasterizer(cfg, dm)
dataset = EgoDataset(cfg, zarr_dataset, rast)
scene_idx = 2
indexes = dataset.get_scene_indices(scene_idx)
images = []

for idx in indexes:
    
    data = dataset[idx]
    im = data["image"].transpose(1, 2, 0)
    im = dataset.rasterizer.to_rgb(im)
    target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
    center_in_pixels = np.asarray(cfg["raster_params"]["ego_center"]) * cfg["raster_params"]["raster_size"]
    draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)
    clear_output(wait=True)
    display(PIL.Image.fromarray(im[::-1]))

というわけで、この一枚の画像にはたくさんの情報が詰まっています。頑張って指摘していきますが、何か間違いがあったら知らせてください。では、画像を分解してみましょう。
+ ここに4つの道路が交差しています。
+ 緑のブロブは自動運転車の動きを表していますが、サンプルとしてこのような交通状況での自動運転車の動きを予測する必要があります。

このデータの詳細がわからないと、他にどんな推論ができるのかよくわかりませんので、これらの画像を見てみましょう。

In [ ]:
import numpy as np
from IPython.display import display, clear_output
import PIL
 
cfg["raster_params"]["map_type"] = "py_satellite"
rast = build_rasterizer(cfg, dm)
dataset = EgoDataset(cfg, zarr_dataset, rast)
scene_idx = 2
indexes = dataset.get_scene_indices(scene_idx)
images = []

for idx in indexes:
    
    data = dataset[idx]
    im = data["image"].transpose(1, 2, 0)
    im = dataset.rasterizer.to_rgb(im)
    target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
    center_in_pixels = np.asarray(cfg["raster_params"]["ego_center"]) * cfg["raster_params"]["raster_size"]
    draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)
    clear_output(wait=True)
    display(PIL.Image.fromarray(im[::-1]))

単純なプロットよりもはるかに詳細な分析を可能にします。私は次のような推論をすると思います。
+ 緑は自動運転車を表し、青は主に我々が予測する必要がある他のすべての車/車両/外因性因子を表しています。
+ 私の仮説では、青は車両が通過する必要がある経路を表しています。
+ 車両が通過する経路を正確に予測できれば、自動運転車がその場で軌道を計算するのが容易になります。

また、乗り物の全体的な動きも見てみたいものです。

In [ ]:
from IPython.display import display, clear_output
from IPython.display import HTML

import PIL
import matplotlib.pyplot as plt
from matplotlib import animation, rc
def animate_solution(images):

    def animate(i):
        im.set_data(images[i])
 
    fig, ax = plt.subplots()
    im = ax.imshow(images[0])
    
    return animation.FuncAnimation(fig, animate, frames=len(images), interval=60)
cfg["raster_params"]["map_type"] = "py_satellite"
rast = build_rasterizer(cfg, dm)
dataset = EgoDataset(cfg, zarr_dataset, rast)
scene_idx = 34
indexes = dataset.get_scene_indices(scene_idx)
images = []

for idx in indexes:
    
    data = dataset[idx]
    im = data["image"].transpose(1, 2, 0)
    im = dataset.rasterizer.to_rgb(im)
    target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
    center_in_pixels = np.asarray(cfg["raster_params"]["ego_center"]) * cfg["raster_params"]["raster_size"]
    draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)
    clear_output(wait=True)
    images.append(PIL.Image.fromarray(im[::-1]))
anim = animate_solution(images)
HTML(anim.to_jshtml())

ということで、他の車両の動きと自動運転車の動きのデモです。自動運転車は現在、その動きの中では直線的な道しか取っておらず、直線的な道は他の車両の動きと配置との関係で論理的に見えます。

In [ ]:
from IPython.display import display, clear_output
from IPython.display import HTML

import PIL
import matplotlib.pyplot as plt
from matplotlib import animation, rc
def animate_solution(images):

    def animate(i):
        im.set_data(images[i])
 
    fig, ax = plt.subplots()
    im = ax.imshow(images[0])
    
    return animation.FuncAnimation(fig, animate, frames=len(images), interval=60)
cfg["raster_params"]["map_type"] = "py_semantic"
rast = build_rasterizer(cfg, dm)
dataset = EgoDataset(cfg, zarr_dataset, rast)
scene_idx = 34
indexes = dataset.get_scene_indices(scene_idx)
images = []

for idx in indexes:
    
    data = dataset[idx]
    im = data["image"].transpose(1, 2, 0)
    im = dataset.rasterizer.to_rgb(im)
    target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
    center_in_pixels = np.asarray(cfg["raster_params"]["ego_center"]) * cfg["raster_params"]["raster_size"]
    draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)
    clear_output(wait=True)
    images.append(PIL.Image.fromarray(im[::-1]))
anim = animate_solution(images)
HTML(anim.to_jshtml())

また、Lyftのレベル5キットのセマンティックオプションを使うことで、より簡易な動きができるようになりました。

In [ ]:
from IPython.display import display, clear_output
import PIL
 
cfg["raster_params"]["map_type"] = "py_semantic"
rast = build_rasterizer(cfg, dm)
dataset = EgoDataset(cfg, zarr_dataset, rast)
scene_idx = 34
indexes = dataset.get_scene_indices(scene_idx)
images = []

for idx in indexes:
    
    data = dataset[idx]
    im = data["image"].transpose(1, 2, 0)
    im = dataset.rasterizer.to_rgb(im)
    target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
    center_in_pixels = np.asarray(cfg["raster_params"]["ego_center"]) * cfg["raster_params"]["raster_size"]
    draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)
    clear_output(wait=True)
    images.append(PIL.Image.fromarray(im[::-1]))
    
anim = animate_solution(images)
HTML(anim.to_jshtml())

あまりクラスタ化されていないビューとしては良いのですが、より詳細で高レベルなデータの概要を知りたい場合は、セマンティックビューを使用してみると良いでしょう。

さて、エージェントの観点からはどうでしょうか？これまでのところ、ほとんどの公開ノートブックでは、主にエージェントの視点からモデリングを行っていますので、これを検討するのは非常に興味深いことでしょう。

In [ ]:
import numpy as np
from IPython.display import display, clear_output
import PIL
 
cfg["raster_params"]["map_type"] = "py_satellite"
rast = build_rasterizer(cfg, dm)
dataset = AgentDataset(cfg, zarr_dataset, rast)
scene_idx = 2
indexes = dataset.get_scene_indices(scene_idx)
images = []

for idx in indexes:
    
    data = dataset[idx]
    im = data["image"].transpose(1, 2, 0)
    im = dataset.rasterizer.to_rgb(im)
    target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
    center_in_pixels = np.asarray(cfg["raster_params"]["ego_center"]) * cfg["raster_params"]["raster_size"]
    draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)
    clear_output(wait=True)
    display(PIL.Image.fromarray(im[::-1]))

これらをGIFとして保存して、エージェントの動きを可視化した方がいいかもしれません。これをもっとシンプルな形で試してみて、エージェントのデータセットにセマンティックビューを使ってみましょう。

In [ ]:
import numpy as np
from IPython.display import display, clear_output
import PIL
 
cfg["raster_params"]["map_type"] = "py_semantic"
rast = build_rasterizer(cfg, dm)
dataset = AgentDataset(cfg, zarr_dataset, rast)
scene_idx = 2
indexes = dataset.get_scene_indices(scene_idx)
images = []

for idx in indexes:
    
    data = dataset[idx]
    im = data["image"].transpose(1, 2, 0)
    im = dataset.rasterizer.to_rgb(im)
    target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
    center_in_pixels = np.asarray(cfg["raster_params"]["ego_center"]) * cfg["raster_params"]["raster_size"]
    draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)
    clear_output(wait=True)
    display(PIL.Image.fromarray(im[::-1]))

また、matplotlib型の視点から全体像を捉えることもできます。 [この素晴らしいノートブックを引用いたします。](https://www.kaggle.com/t3nyks/lyft-working-with-map-api)

In [ ]:
from l5kit.data.map_api import MapAPI
from l5kit.rasterization.rasterizer_builder import _load_metadata

semantic_map_filepath = dm.require(cfg["raster_params"]["semantic_map_key"])
dataset_meta = _load_metadata(cfg["raster_params"]["dataset_meta_key"], dm)
world_to_ecef = np.array(dataset_meta["world_to_ecef"], dtype=np.float64)

map_api = MapAPI(semantic_map_filepath, world_to_ecef)
MAP_LAYERS = ["junction", "node", "segment", "lane"]


def element_of_type(elem, layer_name):
    return elem.element.HasField(layer_name)


def get_elements_from_layer(map_api, layer_name):
    return [elem for elem in map_api.elements if element_of_type(elem, layer_name)]


class MapRenderer:
    
    def __init__(self, map_api):
        self._color_map = dict(drivable_area='#a6cee3',
                               road_segment='#1f78b4',
                               road_block='#b2df8a',
                               lane='#474747')
        self._map_api = map_api
    
    def render_layer(self, layer_name):
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_axes([0, 0, 1, 1])
        
    def render_lanes(self):
        all_lanes = get_elements_from_layer(self._map_api, "lane")
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_axes([0, 0, 1, 1])
        for lane in all_lanes:
            self.render_lane(ax, lane)
        return fig, ax
        
    def render_lane(self, ax, lane):
        coords = self._map_api.get_lane_coords(MapAPI.id_as_str(lane.id))
        self.render_boundary(ax, coords["xyz_left"])
        self.render_boundary(ax, coords["xyz_right"])
        
    def render_boundary(self, ax, boundary):
        xs = boundary[:, 0]
        ys = boundary[:, 1] 
        ax.plot(xs, ys, color=self._color_map["lane"], label="lane")
        
        
renderer = MapRenderer(map_api)
fig, ax = renderer.render_lanes()

In [ ]:
def visualize_rgb_image(dataset, index, title="", ax=None):
    """Visualizes Rasterizer's RGB image"""
    data = dataset[index]
    im = data["image"].transpose(1, 2, 0)
    im = dataset.rasterizer.to_rgb(im)

    if ax is None:
        fig, ax = plt.subplots()
    if title:
        ax.set_title(title)
    ax.imshow(im[::-1])
# Prepare all rasterizer and EgoDataset for each rasterizer
rasterizer_dict = {}
dataset_dict = {}

In [ ]:


rasterizer_type_list = ["py_satellite", "satellite_debug", "py_semantic", "semantic_debug", "box_debug", "stub_debug"]

for i, key in enumerate(rasterizer_type_list):
    # print("key", key)
    cfg["raster_params"]["map_type"] = key
    rasterizer_dict[key] = build_rasterizer(cfg, dm)
    dataset_dict[key] = EgoDataset(cfg, zarr_dataset, rasterizer_dict[key])
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()
for i, key in enumerate(["stub_debug", "satellite_debug", "semantic_debug", "box_debug", "py_satellite", "py_semantic"]):
    visualize_rgb_image(dataset_dict[key], index=0, title=f"{key}: {type(rasterizer_dict[key]).__name__}", ax=axes[i])
fig.show()

# メタデータの探索

画像を探索することができるようになったので、ZARRファイルについても少し詳しく調べてみましょう。Pythonライブラリを使って探索するのはかなり簡単で、特にNumPyとの相互運用性があるという点では、とても便利です。

In [ ]:
print("scenes", zarr_dataset.scenes)
print("scenes[0]", zarr_dataset.scenes[0])

また、ChunkedDatasetを使ってシーンのCSVファイルを生成することもできます。

In [ ]:
import pandas as pd
scenes = zarr_dataset.scenes
scenes_df = pd.DataFrame(scenes)
scenes_df.columns = ["data"]; features = ['frame_index_interval', 'host', 'start_time', 'end_time']
for i, feature in enumerate(features):
    scenes_df[feature] = scenes_df['data'].apply(lambda x: x[i])
scenes_df.drop(columns=["data"],inplace=True)
print(f"scenes dataset: {scenes_df.shape}")
scenes_df.head()

しかし、私たちはこれで十分なのでしょうか？いいえ、そうではありません。今後はKkkillerのデータセットを使って、さらに表形式のデータを探っていこうと思います。

In [ ]:
agents = pd.read_csv('../input/lyft-motion-prediction-autonomous-vehicles-as-csv/agents_0_10019001_10019001.csv')
agents

ここには、以下のようなおなじみの機能を含む、私たちのために使用できる豊富な情報があります。
1. x, y, および z の共線性
2. 偏走
3. その他の外部要因の確率

In [ ]:
import seaborn as sns
colormap = plt.cm.magma
cont_feats = ["centroid_x", "centroid_y", "extent_x", "extent_y", "extent_z", "yaw"]
plt.figure(figsize=(16,12));
plt.title('Pearson correlation of features', y=1.05, size=15);
sns.heatmap(agents[cont_feats].corr(),linewidths=0.1,vmax=1.0, square=True, 
            cmap=colormap, linecolor='white', annot=True);


ここでは、 **centroid_x** と **centroid_y** という変数には強い負の相関があり、最も強い相関は **extent_z** と **extent_x** の間のもので、0.4 となっています。また、この問題の代替的なアプローチとして、kkiller氏が彼の素晴らしいカーネルで実証したように、XGBoost/LightGBMモデルを使ってみることもできます。

### centroid_x と centroid_y

In [ ]:
import seaborn as sns
plot = sns.jointplot(x=agents['centroid_x'][:1000], y=agents['centroid_y'][:1000], kind='hexbin', color='blueviolet')
plot.set_axis_labels('center_x', 'center_y', fontsize=16)

plt.show()

2つのセントロイドがやや強い負の相関と、一見似たような変数分布を持っているように見えます。両方の変数の間に負の相関があるようです。

In [ ]:
fig = plt.figure(figsize=(15, 15));
sns.distplot(agents['centroid_x'], color='steelblue');
sns.distplot(agents['centroid_y'], color='purple');
plt.title("Distributions of Centroid X and Y");

centroid_xの右に歪んだ分布は、 centroid_yのそれよりもかなり極端なようです。どちらの分布も非常に似ていません。

### extent_x, extent_y と extent_z

In [ ]:
fig = plt.figure(figsize=(15, 15));
sns.distplot(agents['extent_x'], color='steelblue');
sns.distplot(agents['extent_y'], color='purple');

plt.title("Distributions of Extents X and Y");

エクステントXとエクステントYの分布は、セントロイドXと同様に大きく右に傾いているように見えますが、プロットの読みやすさのためにエクステントZを省いていますので、それを見てみましょう。

データを滑らかにします。

In [ ]:
fig = plt.figure(figsize=(15, 15));
sns.distplot(agents['extent_z'], color='steelblue');

plt.title("Distributions of Extents z");

もう一度言いますが、すべての `extent` 変数と同じように、右に歪んだ分布を持っています。

### 偏走

In [ ]:
fig = plt.figure(figsize=(15, 15));
sns.distplot(agents['yaw'], color='steelblue');

plt.title("Distributions of Extents z");

この分布は、私がそれらを呼ぶように、いくつかの "突起 "を持っているように見えます。これで、我々の目的がどれだけ実現可能かを確認するために、フレームデータの探索に移ることができるようになりました。

In [ ]:
frms = pd.read_csv("../input/lyft-motion-prediction-autonomous-vehicles-as-csv/frames_0_124167_124167.csv")
frms.head()

ここには、セントロイドに関する回転がありますが、これは非常に興味深い考察になるでしょう。これらの変数を一度に複数チェックする必要がありそうです。

### 自己回転

まず第一に、それぞれに対応する9つの自己回転列があります。そこで、より高度な分析に移る前に、これらの変数の相関関係を簡単にチェックしたいと思います。

In [ ]:
import seaborn as sns
colormap = plt.cm.magma
cont_feats = ["ego_rotation_xx", "ego_rotation_xy", "ego_rotation_xz", "ego_rotation_yx", "ego_rotation_yy", "ego_rotation_yz", "ego_rotation_zx", "ego_rotation_zy", "ego_rotation_zz"]
plt.figure(figsize=(16,12));
plt.title('Pearson correlation of features', y=1.05, size=15);
sns.heatmap(frms[cont_feats].corr(),linewidths=0.1,vmax=1.0, square=True, 
            cmap=colormap, linecolor='white', annot=True);


この相関分析から注意すべきこと
1. y` と `z` の回転座標はほとんどの場合、相関がないように見える。
2. `x` を持つ座標は z 次元の回転と強く相関している (これは何かを示しているのだろうか？)

### 二値特徴

In [ ]:
import numpy as np
zero_count_list, one_count_list = [], []
cols_list = ["label_probabilities_PERCEPTION_LABEL_UNKNOWN","label_probabilities_PERCEPTION_LABEL_CAR","label_probabilities_PERCEPTION_LABEL_CYCLIST","label_probabilities_PERCEPTION_LABEL_PEDESTRIAN"]
for col in cols_list:
    zero_count_list.append((agents[col]==0).sum())
    one_count_list.append((agents[col]==1).sum())

N = len(cols_list)
ind = np.arange(N)
width = 0.35

plt.figure(figsize=(6,10))
p1 = plt.barh(ind, zero_count_list, width, color='purple')
p2 = plt.barh(ind, one_count_list, width, left=zero_count_list, color="steelblue")
plt.yticks(ind, cols_list)
plt.legend((p1[0], p2[0]), ('Zero count', 'One Count'))
plt.show()

d3.jsを使ったバイナリ機能のクイックチェック - グラフィックのレンダリングに時間がかかるかもしれません。

In [ ]:
""""""
import json, random
def parse_demo(col):
    bigtxt = ";".join(df[col].dropna())
    wrds = bigtxt.split(";")
    wrds = Counter(wrds).most_common()
    return wrds 

strr = "id,value,color\nAudience Demographics,\n"
demographics = ['Gender', 'Age', 'SexualOrientation', 'RaceEthnicity', 'EducationParents', 'Dependents']
#demographics = ['Gender', 'Age', 'SexualOrientation', 'RaceEthnicity', 'EducationParents']
colors = ['#5b9aff', '#ff77bd', '#82ff8a', '#9b9493', '#5b9aff', '#ff77bd', '#82ff8a', '#9b9493']
for i,col in enumerate(demographics):
    strr += "Audience Demographics." + col + ",\n"
    
    response = parse_demo(col)
    total = sum([x[1] for x in response])
    for term in response:
        cent = float(term[1])*100 / total
        strr += "Audience Demographics." + col +"."+ term[0].split("(")[0].replace(",","") +","+ str(cent) + ","+colors[i]+"\n"

fout = open("tomdata.csv", "w")

fout.write(strr)



html2 =""" <style>
.link {
        fill: none;
        stroke: #555;
        stroke-opacity: 0.4;
        stroke-width: 1px;
    }
    text {
        font-family: "Arial Black", Gadget, sans-serif;
        fill: black;
        font-weight: bold;
        font-size: 14px
    }

    .xAxis .tick text{
        fill: black;
    }
    
    .grid .tick line{
        stroke: grey;
        stroke-dasharray: 5, 10;
        opacity: 0.7;
    }
    .grid path{
        stroke-width: 0;
    }

    .node1 circle {
        fill: #999;
    }
    .node1--internal circle {
        fill: #555;
    }
    .node1--internal text {
        font-size: 16px;
        text-shadow: 0 2px 0 #fff, 0 -2px 0 #fff, 2px 0 0 #fff, -2px 0 0 #fff;
    }
    .node1--leaf text {
        fill: white;
    }
    .ballG text {
        fill: white;
    }

    .shadow {
        -webkit-filter: drop-shadow( -1.5px -1.5px 1.5px #000 );
        filter: drop-shadow( -1.5px -1.5px 1.5px #000 );
    }</style>
    <body>
    <br><br>
    <svg id="five" width="900" height="1200"></svg>
    <br><br><br>
</body>
"""


js2 = """
 
 require(["d3"], function(d3) {
  
    var svg1 = d3.select("#five"),
            width = +svg1.attr("width"),
            height = +svg1.attr("height"),
            g1 = svg1.append("g").attr("transform", "translate(20,10)");       // move right 20px.
            
    var xScale =  d3.scaleLinear()
            .domain([0,100])
            .range([0, 400]);

    var xAxis = d3.axisTop()
            .scale(xScale);

    // Setting up a way to handle the data
    var tree1 = d3.cluster()                 // This D3 API method setup the Dendrogram datum position.
            .size([height, width - 550])    // Total width - bar chart width = Dendrogram chart width
            .separation(function separate(a, b) {
                return a.parent == b.parent            // 2 levels tree1 grouping for category
                || a.parent.parent == b.parent
                || a.parent == b.parent.parent ? 0.4 : 0.8;
            });

    var stratify = d3.stratify()            // This D3 API method gives cvs file flat data array dimensions.
            .parentId(function(d) { return d.id.substring(0, d.id.lastIndexOf(".")); });

# ベースライン・モデル (ソース: [こちら](https://github.com/lyft/l5kit/blob/master/examples/agent_motion_prediction/agent_motion_prediction.ipynb) と [こちら](https://www.kaggle.com/pestipeti/pytorch-baseline-inference))

これは主に私がLyftのベースラインモデルを使って、提供されたデータセットフォーマットでPyTorchモデルにどのようにフィットするかを実演するため、トレーニングしているものです。

また、PyTorch Lightningとそれに付随するすべての利点を使用しています。PL-Lightningについては、後で詳しく説明します。

In [ ]:
from typing import Dict
!pip install pytorch-lightning

from tempfile import gettempdir
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.models.resnet import resnet18
from tqdm import tqdm
from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.rasterization import build_rasterizer
from l5kit.evaluation import write_pred_csv, compute_metrics_csv, read_gt_csv, create_chopped_dataset
from l5kit.evaluation.chop_dataset import MIN_FUTURE_STEPS
from l5kit.evaluation.metrics import neg_multi_log_likelihood, time_displace
from l5kit.geometry import transform_points
from l5kit.visualization import PREDICTED_POINTS_COLOR, TARGET_POINTS_COLOR, draw_trajectory
from prettytable import PrettyTable
from pathlib import Path
import pytorch_lightning as pl
import os
cfg = load_config_data('../input/lyft-config-files/agent_motion_config.yaml')
class Mod(torch.nn.Module):
    def __init__(self, cfg: Dict):
        super(Mod, self).__init__()
        self.backbone = resnet18(pretrained=False)
        
        num_history_channels = (cfg["model_params"]["history_num_frames"] + 1) * 2
        num_in_channels = 3 + num_history_channels

        self.backbone.conv1 = nn.Conv2d(
            num_in_channels,
            self.backbone.conv1.out_channels,
            kernel_size=self.backbone.conv1.kernel_size,
            stride=self.backbone.conv1.stride,
            padding=self.backbone.conv1.padding,
            bias=False,
        )
        
        # This is 512 for resnet18 and resnet34;
        # And it is 2048 for the other resnets
        backbone_out_features = 512

        # X, Y coords for the future positions (output shape: Bx50x2)
        num_targets = 2 * cfg["model_params"]["future_num_frames"]

        # You can add more layers here.
        self.head = nn.Sequential(
            # nn.Dropout(0.2),
            nn.Linear(in_features=backbone_out_features, out_features=4096),
        )

        self.logit = nn.Linear(4096, out_features=num_targets)
    def forward(self):
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)

        x = self.backbone.avgpool(x)
        x = torch.flatten(x, 1)
        
        x = self.head(x)
        x = self.logit(x)
        
        return x        

def forward(data, model, device, criterion):
    inputs = data["image"].to(device)
    target_availabilities = data["target_availabilities"].unsqueeze(-1).to(device)
    targets = data["target_positions"].to(device)
    # Forward pass
    outputs = model(inputs).reshape(targets.shape)
    loss = criterion(outputs, targets)
    # not all the output steps are valid, but we can filter them out from the loss using availabilities
    loss = loss * target_availabilities
    loss = loss.mean()
    return loss, outputs

class LightningLyft(pl.LightningModule):
    def __init__(self, model):
        super(LightningLyft, self).__init__()
        self.model = model
        
    def forward(self, x, *args, **kwargs):
        return self.model(x)
    
    def prepare_train_data(self):
        train_cfg = cfg["train_data_loader"]
        rasterizer = build_rasterizer(cfg, dm)
        train_zarr = ChunkedDataset(dm.require(train_cfg["key"])).open()
        train_dataset = AgentDataset(cfg, train_zarr, rasterizer)
        train_dataloader = DataLoader(train_dataset, shuffle=train_cfg["shuffle"], batch_size=train_cfg["batch_size"], 
                             num_workers=train_cfg["num_workers"])
        return train_dataloader
            
    def training_step(self, batch, batch_idx):
        tr_it = iter(train_dataloader)
        progress_bar = tqdm(range(cfg["train_params"]["max_num_steps"]))
        losses_train = []
        model = self.model
        for n in [0, 1, 2 , 3, 4]:
            try:
                data = next(tr_it)
            except StopIteration:
                tr_it = iter(train_dataloader)
                data = next(tr_it)
            model.train()
            torch.set_grad_enabled(True)
            device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
            optimizer = optim.Adam(model.parameters(), lr=1e-3)
            criterion = nn.MSELoss(reduction="none")
            loss, _ = forward(data, model, device, criterion)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            losses_train.append(loss.item())
            print(f"LOSS FOR EPOCH {n}: {loss.item()}")
            
    def configure_optimizers(self):
        optimizer = optim.Adam(model.parameters(), lr=1e-3)
        return optimizer

In [ ]:
# ===== INIT DATASET
train_cfg = cfg["train_data_loader"]
rasterizer = build_rasterizer(cfg, dm)
train_zarr = ChunkedDataset(dm.require(train_cfg["key"])).open()
train_dataset = AgentDataset(cfg, train_zarr, rasterizer)
train_dataloader = DataLoader(train_dataset, shuffle=train_cfg["shuffle"], batch_size=train_cfg["batch_size"], 
                             num_workers=train_cfg["num_workers"])

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = Mod(cfg).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss(reduction="none")

In [ ]:
# ==== TRAIN LOOP
res = []
tr_it = iter(train_dataloader)
model = LightningLyft(build_model(cfg))
progress_bar = tqdm(range(cfg["train_params"]["max_num_steps"]))
losses_train = []
for _ in progress_bar:
    try:
        data = next(tr_it)
    except StopIteration:
        tr_it = iter(train_dataloader)
        data = next(tr_it)
    model.train()
    torch.set_grad_enabled(True)
    loss, _ = forward(data, model, device, criterion)
    res.append(_)
    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    losses_train.append(loss.item())
    progress_bar.set_description(f"loss: {loss.item()} loss(avg): {np.mean(losses_train)}")

PyTorch lightningを使ったモデルの学習は初めてなので、かなり満足しています。しかし今は、このモデルが具体的に何を学習したのかを見てみたいと思います。<br>予測値を結果用の配列 `res` に保存することができました。学習したことを確認するのは後になりますが、今後は予測を進めていきます。

ソース: https://github.com/lyft/l5kit/blob/master/examples/visualisation/visualise_data.ipynb